In [1]:
import numpy as np
import tifffile
import xml.etree.ElementTree as ET
import tifffile
import glob
import pandas as pd
import numpy as np
import napari
import dask.array as da
from dask.array.image import imread
import cv2
from matplotlib.path import Path

In [2]:
viewer = napari.Viewer()

In [3]:
data_directory = 'D:/tmp/rahul/parent/_Tray01_Slide02.vsi.Collection/'

In [4]:
def get_values(folder):
    tree = ET.parse(folder + 'Layer-source-metadata.xml')
    tiffs = glob.glob(folder + 'CH*_Z.tif')
    tiffs.sort()
    #img = np.array([tifffile.imread(tiff) for tiff in tiffs])
    img = imread(folder + 'CH*_Z.tif')
    root = tree.getroot()
    for elem in root.iter():
        if ('ImageAxis0Origin' in elem.tag):
            x0 = float((elem.attrib['value']).split(' ')[0])
        if ('ImageAxis1Origin' in elem.tag):
            y0 = float((elem.attrib['value']).split(' ')[0])
        if ('ImageAxis0Resolution' in elem.tag):
            xres = float((elem.attrib['value']).split(' ')[0])
    return img.max(axis=1), x0, y0, xres

def get_all_folders(parent_folder):
    imgs = []
    vals = []
    fnames = glob.glob(parent_folder + '/*Layer*')
    for folder in fnames:
        img, x0, y0, xres = get_values(folder+'/')
        imgs.append(img)
        vals.append([x0, y0, xres])
    vals = np.array(vals)
    x = (vals[:,0] - np.min(vals[:,0]))*1E6/np.median(vals[:,2])
    y = (vals[:,1] - np.min(vals[:,1]))*1E6/np.median(vals[:,2])

    x = x.astype(int)
    y = y.astype(int)

    xx = y
    y = x
    x = xx
    return imgs, x, y



In [5]:
def get_order_of_images(lines):
    pts_list = []
    for lin in lines:
        for a in range(0, len(lin)-1):
            point1 = lin[a]
            point2 = lin[a+1]
            points = make_points_along_line(point1, point2, number=30)
            pts_list = pts_list + points.tolist()

    pts_list = np.array(pts_list)
    positions = []
    #for pt in viewer.layers[-1].data:
    for pt in pts_list:
        df_pos = -1
        z = pt[0]
        rois = np.array(viewer.layers[-2].data)
        c_rois = rois[rois[:,:,0]==z].reshape(-1,4,3)[:,:,[1,2]]
        for a, rec in enumerate(c_rois):
            path = Path(rec)
            contained=path.contains_points([pt[1:3]])
            if np.any(contained):
                df_pos = a
        positions.append([z,df_pos])
    positions=np.array(positions)
    positions = positions.astype(int)

    read_list = []
    for p in positions:
        if p[1] != -1:
            read_list.append(p[1])

    read_list = pd.unique(read_list)
    return read_list

def backsub(inp):
    filterSize =(60, 60)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                    filterSize)
    blurred = cv2.GaussianBlur(inp, (5, 5), 0)
    tophat_img = cv2.morphologyEx(blurred,
                                cv2.MORPH_TOPHAT,
                                kernel)
    rtn = inp.astype(np.single) - (blurred-tophat_img)
    rtn = np.clip(rtn, 0, np.inf)

    return rtn

def make_points_along_line(point1, point2, number=10):
    pt_list = []
    delpt = point2-point1
    dist = np.sqrt(np.dot(delpt, delpt).sum())
    delpt = delpt / dist
    spacing = dist/number
    for i in range(0, number+1):
        pt_list.append(point1 + i*spacing*delpt)
    return np.array(pt_list)


In [6]:
imgs, x, y = get_all_folders(data_directory)

In [7]:
x_max = np.max(np.array([i.shape[1] for i in imgs]))
y_max = np.max(np.array([i.shape[2] for i in imgs]))


In [8]:
downsample = 4

In [9]:
xdims = np.max(x) + x_max
ydims = np.max(y) + y_max

xdims = np.ceil(xdims/downsample).astype(int)
ydims = np.ceil(ydims/downsample).astype(int)

In [10]:
master_img = np.zeros((imgs[0].shape[0], xdims, ydims))

In [11]:
for idx, img in enumerate(imgs):
    ds_img = img[:,::downsample,::downsample]
    xstart = np.floor(x[idx]/downsample).astype(int)
    ystart = np.floor(y[idx]/downsample).astype(int)
    xrng = ds_img.shape[1]
    yrng = ds_img.shape[2]
    master_img[:,xstart:xstart+xrng, ystart:ystart+yrng] = ds_img

In [12]:
viewer.layers.clear()
viewer.add_image(master_img, blending='additive', channel_axis=0, contrast_limits=[0,20000])

[<Image layer 'Image' at 0x27bf96d1ab0>,
 <Image layer 'Image [1]' at 0x27bf96d2ec0>,
 <Image layer 'Image [2]' at 0x27bf4d700d0>,
 <Image layer 'Image [3]' at 0x27bf1099a20>]

In [13]:
# Make ROI boxes
boxes = []
for i, img in enumerate(imgs):
    x_min = int(x[i]/downsample)
    y_min = int(y[i]/downsample)
    x_max = int(img.shape[1]/downsample) + x_min
    y_max = int(img.shape[2]/downsample) + y_min
    boxes.append([[0, x_min, y_min], [0, x_max, y_min], [0, x_max, y_max], [0, x_min, y_max]])

In [14]:
viewer.add_shapes(boxes, opacity=0.2)

<Shapes layer 'boxes' at 0x27bde7897e0>


# Draw lines here

In [15]:
lines = viewer.layers[-1].data
read_list = get_order_of_images(lines)

for idx, i in enumerate(read_list):
    #viewer.add_image(self.imgs[i], channel_axis=0, contrast_limits=[0,20000])
    back_subbed = np.array([backsub(img.compute()) for img in imgs[i]])
    tifffile.imwrite(data_directory+'/outputs/'+'{:0>2}'.format(idx) + '.tif', back_subbed, imagej=True, metadata={'axes':'CYX', 'mode':'color'})
        

C:\Users\smc\AppData\Local\Temp\ipykernel_6380\1049602390.py:32: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  read_list = pd.unique(read_list)
